In [18]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np

from dicodile.utils.csc import reconstruct
from dicodile.utils.viz import display_dictionaries

In [7]:
data_dir_path = Path.home() / "data/pattern_detection_tokam"

In [8]:
input_dir_path = data_dir_path / "input"
file_path_list = sorted(input_dir_path.glob("*"))

In [9]:
for file_path in file_path_list:
    break
    plt.imshow(plt.imread(file_path))
    plt.show()

In [ ]:
image_array = plt.imread(file_path_list[0])
plt.imshow(image_array)
plt.show()

Dictionary shape: (n_atoms, n_channels, *atom_support)

In [ ]:
learned_dict_dir_path = data_dir_path / "learned_dictionaries/first_image"

time_str_list = sorted(
    {
        f"{file_path.stem.split('_')[2]}_{file_path.stem.split('_')[3]}"
        for file_path in learned_dict_dir_path.glob("*")
    }
)

print(time_str_list)

In [39]:
time_str = time_str_list[-1]

D_hat = np.load(learned_dict_dir_path / f"D_hat_{time_str}.npy")
z_hat = np.load(learned_dict_dir_path / f"z_hat_{time_str}.npy")

In [ ]:
print(D_hat.shape, D_hat.min(), D_hat.max())

In [ ]:
print(np.sum(D_hat**2, axis=(1, 2, 3)))

In [ ]:
normalisation = np.reshape(np.max(D_hat, axis=(1, 2, 3)), (-1, 1, 1, 1))

display_dictionaries(D_hat / normalisation)
plt.show()

In [ ]:
print(z_hat.shape, z_hat.min(), z_hat.max())

In [ ]:
plt.imshow(z_hat.transpose([1, 2, 0]))
plt.show()

In [45]:
X_hat = reconstruct(z_hat, D_hat)
X_hat = np.clip(X_hat, 0, 1)
X_hat = X_hat.transpose([1, 2, 0])

In [ ]:
f, (ax1, ax2) = plt.subplots(2, 1, figsize=[6.4, 8])

ax1.imshow(image_array, cmap='gray')
ax1.set_title('Original image')
ax1.axis('off')

ax2.imshow(X_hat, cmap='gray')
ax2.set_title('Recovered image')
ax2.axis('off')
plt.tight_layout()